# Walkthrough Example

### Imports

In [ ]:
# ruff: noqa: T201
import ipywidgets as widgets
import trimesh
from numpy.typing import NDArray

from edge_mender import EdgeMender
from edge_mender.data_factory import DataFactory
from edge_mender.mesh_generator import MeshGenerator
from edge_mender.visualizer import Visualizer

### Create the test data

In [ ]:
dataset = widgets.Dropdown(
    options=[
        ("Simple Extrusion", DataFactory.simple_extrusion),
        ("Double Extrusion", DataFactory.double_extrusion),
        ("Triple Extrusion", DataFactory.triple_extrusion),
        ("Stairs", DataFactory.stairs),
        ("Ceiling", DataFactory.ceiling),
        ("Double Tower Ceiling", DataFactory.double_tower_ceiling),
        ("Hanging Points", DataFactory.hanging_points),
        ("Checkerboard", DataFactory.checkerboard),
        ("Hole", DataFactory.hole),
        ("Kill You", DataFactory.kill_you),
        ("Random", DataFactory.random),
    ],
    description="Dataset:",
)
dataset

In [ ]:
meshing_algorithm = widgets.Dropdown(
    options=[
        ("Cuberille", MeshGenerator.to_mesh_cuberille),
        ("Surface Nets", MeshGenerator.to_mesh_surface_nets),
        ("Dual Contouring", MeshGenerator.to_mesh_dual_contouring),
    ],
    description="Dataset:",
)
meshing_algorithm

In [ ]:
# Create test voxel grid data
data: NDArray = dataset.value()

# Convert the data to a mesh
mesh: trimesh.Trimesh = meshing_algorithm.value(data)

# Visualize the mesh
visualizer = Visualizer.show_mesh(mesh)

### Find the non-manifold edges

In [ ]:
# Find the non-manifold edges
mender = EdgeMender(mesh)
faces, vertices, edges = mender.find_non_manifold_edges()

# Print the mesh information
print(f"Volume: {mesh.volume}")
print(f"Watertight: {mesh.is_watertight}")
print(f"Non-manifold edges: {len(edges)}")

# Visualize the non-manifold edges
plotter = Visualizer.show_mesh(
    mesh,
    highlight_faces=faces.flatten().tolist(),
    highlight_vertices=vertices.flatten().tolist(),
    highlight_edges=vertices.tolist(),
    add_vertex_labels=True,
)

### Repair the mesh

In [ ]:
shift = widgets.Dropdown(
    options=[
        ("Yes", True),
        ("No", False),
    ],
    description="Shift vertices:",
)
shift

In [ ]:
# Find the non-manifold edges
mender = EdgeMender(mesh)
mender.repair(shift_distance=0.1 if shift else 0.0)
remaining_non_manifold_edges = mender.find_non_manifold_edges()[2]

# Print the repaired mesh information
print(f"Volume: {mesh.volume}" + (" (WARNING: Shift applied)" if shift else ""))
print(f"Watertight: {mesh.is_watertight}")
print(f"Remaining non-manifold edges: {len(remaining_non_manifold_edges)}")

# Visualize the repaired mesh
plotter = Visualizer.show_mesh(mesh)